# MODEL-PROJECT: The Black-Scholes(-Merton)-Model

**Before we start please install these packages in your VScode terminal (if not already installed)**

To do that just remove the '#' in the codecell below and run the code once. Be sure to put the '#' back in before you continue as to not run into problems when rerunning the code.

In [18]:
# !pip install yfinance
# !pip install ipywidgets
# !pip install dash
# !pip install dash-core-components

Imports and set magics:

In [19]:
import numpy as np
from scipy import optimize
import sympy as sp
import matplotlib.pyplot as plt
from scipy.stats import norm
import yfinance as yf
from dash import dcc
import ipywidgets as widgets
from IPython.display import display

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Model description

The Black-Scholes model is a mathematical model for pricing European-style options. It's widely used in the financial industry, and it has significantly impacted the field of financial economics. The model assumes that the price of the underlying asset follows a geometric Brownian motion, and it derives a closed-form solution for the price of said asset. It provides a theoretical estimate of the value of a call or put option based on the following parameters:

-The current price of the underlying asset (S)  
-The strike price of the option (K)  
-The time to maturity (T)  
-The risk-free interest rate (r)  
-The volatility of the underlying asset (σ)  

Deriving the Black-Scholes PDE for a call-option on a non-dividend paying stock with strike K and maturity T. We assume that the stock price follows a geometric Brownian motion so that:

$ d (S_t) = μS_t dt + σS_tdW_t $ (i)

where $ W_t $ is a standard Brownian motion. We also assume that interest rates are constant so that 1 unit of currency invested in the cash account at time 0 will be worth $ B_t := exp(rt) $ at time t. We will denote by $ C(S, t) $ the value of the call option at time t. By Itˆo’s lemma we know that:

$ dC(S, t) = μS_t\frac{∂ C}{∂ S} + \frac{∂ C}{∂ t} + \frac{1}{2}σ^2S^2\frac{∂^2C}{∂S^2} dt + σS_t\frac{∂ C}{∂ S}dW_t$ (ii)

Let us now consider a self-financing trading strategy where at each time t we hold $x_t$ units of the cash account and $y_t$ units of the stock. Then $P_t$, the time t value of this strategy satisfies:

$P_t = x_tB_t + y_tS_t $ (iii)

We will choose $x_t$ and $y_t$ in such a way that the strategy replicates the value of the option. The self-financing assumption implies that:

$ dP_t = x_tdB_t + y_tdS_t$  (iv)  
$      = rx_tB_tdt + y_t(μS_tdt + σS_tdW_t) $  
$      = (rx_tB_t + y_tμS_t)dt + y_tσS_tdW_t $ (v)

We can equate terms in (ii)) with the corresponding terms in (v) to obtain:

$ y_t = \frac{∂ C}{∂ S} $  (vi)

$ rx_tB_t = \frac{∂ C}{∂ t} + \frac{1}{2}σ^2S^2\frac{∂^2C}{∂S^2} $  (vii)

If we set $C_0 = P_0$, the initial value of our self-financing strategy, then it must be the case that $C_t = P_t$ for all t since C and P have the same dynamics. This is true by construction after we equated terms in (ii) with the corresponding terms in (v). Substituting (vi) and (vii) into (iii) we obtain:


$ rS_t\frac{∂ C}{∂ S} + \frac{∂ C}{∂ t} + \frac{1}{2}σ^2S^2\frac{∂^2C}{∂S^2} - rC = 0 $ (viii) 


the Black-Scholes PDE. In order to solve (viii) boundary conditions must also be provided. In the case of the call option, these conditions are as follows:

$ C(S, T) = max(S-K, 0), C(0, t) = 0 $ for all t and $C(S, t) ⭢ S $ $as$ $ S ⭢ ∞ $  

We are finally able to arrive at the final formula for a european style call option as the solution to equation (viii):

$ C(S, K, T, r, σ) = S N(d1) - K e^{-rT} N(d2) $ 

where:

$ d1 = \frac{log(\frac{S}{K}) + (r + \frac{σ^2}{2})T}{σ\sqrt{T}} $

$ d2 = d1 - σ\sqrt{T} $  

and $N(x)$ is the cumulative distribution function of the standard normal distribution.

Equivalenty, from what we call the put-call parity we can also easily compute the price of a European put option using the formula for a call. By doing so we arrive at the following:

$ P(S, K, T, r, σ) = K e^{-rT} N(-d2) - S N(-d1) $  
  
  
In this project, we'll implement the Black-Scholes model using Python and perform various analyses, including numerical solutions for different real world assets and visualizations.


## Analytical solution

The Black-Scholes model .....

In [20]:
# Define the Black-Scholes Model symbols
S, K, T, r, sigma = sp.symbols('S K T r sigma')

# Define the Black-Scholes Model equation
d1 = ((r + 0.5*sigma**2)*T + sp.log(S/K)) / (sigma*sp.sqrt(T))
d1

(T*(r + 0.5*sigma**2) + log(S/K))/(sqrt(T)*sigma)

In [21]:
d2 = d1 - sigma*sp.sqrt(T)
d2

-sqrt(T)*sigma + (T*(r + 0.5*sigma**2) + log(S/K))/(sqrt(T)*sigma)

In [22]:
# Define lambdified functions for d1 and d2
d1_lambdified = sp.lambdify((S, K, T, r, sigma), d1, 'numpy')
d2_lambdified = sp.lambdify((S, K, T, r, sigma), d2, 'numpy')

# Define the Black-Scholes Model equation
def black_scholes(S, K, T, r, sigma, option_type='call'):
    d1 = d1_lambdified(S, K, T, r, sigma)
    d2 = d2_lambdified(S, K, T, r, sigma)
    
    if option_type == 'call':
        price = S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    else:
        price = K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)
    
    return price

## Numerical solution

We can also solve the Black-Scholes model numerically using optimization algorithms. In this case, we'll use the Newton-Raphson method.

In [33]:
# Define the initial values for K, S, T, r, and sigma
K = 100
S = 100
T = 1
r = 0.05
sigma = 0.2

# Define a range of strike prices
stock_prices = np.linspace(1, 200, 100)

# Create a function to update the plot with the new value of sigma
def update_plot(K, T, r, sigma):
    call_prices = [black_scholes(S, K, T, r, sigma, option_type='call') for S in stock_prices]
    put_prices = [black_scholes(S, K, T, r, sigma, option_type='put') for S in stock_prices]
    
    plt.figure(figsize=(10, 6))
    plt.plot(stock_prices, call_prices, label="Call prices")
    plt.plot(stock_prices, put_prices, label="Put prices")
    plt.title(f"Black-Scholes Model for K={K}, T={T}, r={r}, sigma={sigma}")
    plt.xlabel("Stock Price")
    plt.ylabel("Option Price")
    plt.legend()
    plt.show()

# Create a slider widget for S
K_slider = widgets.FloatSlider(
    value=K,
    min=S * 0.5,
    max=S * 1.5,
    step=1,
    description='Strike price "K":',
    continuous_update=False
)    

# Create a slider widget for T
T_slider = widgets.FloatSlider(
    value=T,
    min=1,
    max=10,
    step=1,
    description='Time to maturity "T":',
    continuous_update=False
)

# Create a slider widget for r
r_slider = widgets.FloatSlider(
    value=r,
    min=0,
    max=0.2,
    step=0.005,
    description='Risk free rate "r":',
    continuous_update=False
)

# Create a slider widget for sigma
sigma_slider = widgets.FloatSlider(
    value=sigma,
    min=0.01,
    max=1,
    step=0.01,
    description='Sigma:',
    continuous_update=False
)

# Create an output widget to display the plot
out = widgets.Output()

# Display the interactive plot
widgets.interact(update_plot, K=K_slider, T=T_slider, r=r_slider, sigma=sigma_slider)
display(out)


interactive(children=(FloatSlider(value=100.0, continuous_update=False, description='Strike price "K":', max=1…

Output()

# Further analysis

Now, let's analyze how the model changes with different parameter values and visualize the results.

In [24]:
# List of 15 stock tickers 
stock_list = ["AAPL", "MSFT", "GOOG", "AMZN", "TSLA", "NFLX", "BRK-B", "JPM", "JNJ", "V", "WMT", "UNH", "PG", "NVDA", "HD"]
company_list = ["Apple", "Microsoft", "Google", "Amazon", "Tesla", "Netflix", "Berkshire Hathaway", "JPMorgan Chase", "Johnson & Johnson", "Visa", "Walmart", "UnitedHealth Group", "Procter & Gamble", "NVIDIA", "Home Depot"]

# Define dictionary to map companies to tickers
company_ticker_dict = {company: ticker for company, ticker in zip(company_list, stock_list)}

# Function to plot Black-Scholes model for a given stock ticker
def plot_black_scholes(company, T_val=T, r_val=r, sigma_val=sigma):
    with out:
        try:
            # Get stock data
            stock_ticker = company_ticker_dict[company]
            stock_data = yf.download(stock_ticker, progress=False)
            stock_price = stock_data.iloc[-1]["Close"]

            # Calculate call and put prices for a range of strike prices
            strike_prices = np.linspace(stock_price * 0.5, stock_price * 1.5, 100)
            call_prices = [black_scholes(stock_price, K, T_val, r_val, sigma_val, option_type='call') for K in strike_prices]
            put_prices = [black_scholes(stock_price, K, T_val, r_val, sigma_val, option_type='put') for K in strike_prices]

            # Clear the output and create the plot
            out.clear_output(wait=True)
            plt.figure(figsize=(10, 6))
            plt.plot(strike_prices, call_prices, label="Call prices")
            plt.plot(strike_prices, put_prices, label="Put prices")

            plt.title(f"Black-Scholes Model for {company}")
            plt.xlabel("Strike Price")
            plt.ylabel("Option Price")
            plt.legend()
            plt.show()
        except IndexError as e:
            print(f"Error: {e}. Data for {company} might not be available or the stock might be delisted.")


# Create a dropdown menu for company selection
company_dropdown = widgets.Dropdown(
    options=company_list,
    value=company_list[0],
    description="Company:",
)

# Create a slider widget for T
T_slider2 = widgets.FloatSlider(
    value=T,
    min=1,
    max=10,
    step=1,
    description='Time to maturity "T":',
    continuous_update=False
)

# Create a slider widget for r
r_slider2 = widgets.FloatSlider(
    value=r,
    min=0,
    max=0.2,
    step=0.005,
    description='Risk free rate "r":',
    continuous_update=False
)

# Create a slider widget for sigma
sigma_slider2 = widgets.FloatSlider(
    value=sigma,
    min=0.01,
    max=1,
    step=0.01,
    description='Sigma:',
    continuous_update=False
)

# Create an Output widget to display the plot
out = widgets.Output()

# Display the interactive plot
widgets.interact(plot_black_scholes, company=company_dropdown, T_val=T_slider2, r_val=r_slider2, sigma_val=sigma_slider2)
display(out)

interactive(children=(Dropdown(description='Company:', options=('Apple', 'Microsoft', 'Google', 'Amazon', 'Tes…

Output()

In [25]:
# Code we can use to estimate past sigmas: 


# Define a list of ticker symbols for the stocks you want to analyze
tickers = ["AAPL", "MSFT", "AMZN", "GOOGL"]

# Download data for all the tickers in the list
data = yf.download(tickers, start="2022-01-01", end="2023-05-05", group_by="ticker")

# Create a dictionary to store the results
results_dict = {}

# Loop through the list of tickers and calculate the standard deviation for the Close column (if available)
for ticker in tickers:
    # Check if the Close column exists in the data
    if "Close" in data[ticker].columns:
        # Calculate the standard deviation of the Close column
        std_dev = data[ticker]["Close"].std()
    else:
        # If the Close column doesn't exist, set the standard deviation to None
        std_dev = None
        
    results_dict[ticker] = std_dev

# Print the dictionary to display the results
print(results_dict)


[*********************100%***********************]  4 of 4 completed
{'AAPL': 12.740353615867223, 'MSFT': 24.851217339488393, 'AMZN': 24.060345165344298, 'GOOGL': 15.932279210602557}


# Conclusion

Additionally, in this version of the project, we have created an interactive plot to visualize the Black-Scholes option prices for 15 different European stocks. The stocks were chosen to represent a diverse range of industries and countries. The interactive plot allows users to select a stock from a dropdown menu and view the corresponding option prices for the call and put options.